## Crop images and masks

In [18]:
import os
import pathlib as pt
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

#### Define helper functions

In [2]:
def get_dim_tumor(input_mask, show=False):
    """
    Get the bounding box around the tumor
    
    Args:
        input_mask(numpy array): contains the mask image
        show(bool): whether to display the rectangle on the image or not
        
    """
    
    if len(input_mask.shape) == 3:
        input_mask = cv2.cvtColor(input_mask, cv2.COLOR_BGR2GRAY)
    white_area = cv2.findNonZero(input_mask)
    x, y, w, h = cv2.boundingRect(white_area)
    if show:
        print("[INFO] Bbox:", x, y, w, h)
        display_mask = cv2.cvtColor(input_mask, cv2.COLOR_GRAY2RGB)
        cv2.rectangle(display_mask, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imwrite("bbox.jpg", display_mask)
        plt.imshow(display_mask)
    return x, y, w, h

In [3]:
def get_center_rect(x, y, w, h ):
    """
    Given a bounding box return its center
    Args:
        x(int): x-coordinate of the top left corner
        y(int): y-coordinate of the top left corner
        w(int): width
        h(int): height
    
    Returns:
        center(float): center of the rectangle
        
    """
    return (x + w * 0.5, y + h * 0.5)

In [4]:
def get_img_orientation(input_img, show=False):
    """
    Determine the breast position in the image: left or right
    
    Args:
        input_img(numpy array): the image scan
        show(bool): whether to display the image or not
        
    """
    
    width = 150
    pos_left = 10
    pos_right = input_img.shape[1] - 10
    
    rect_left = input_img[:, pos_left:width]
    rect_right = input_img[:, (pos_right - width):pos_right]
    
    m_left = rect_left.mean()
    m_right = rect_right.mean()
    
    if show:
        plt.imshow(rect_left, cmap="gray")
        plt.show()
        plt.imshow(rect_right, cmap="gray")
        plt.show()
        print(m_left, m_right)

    orientation = None
    if m_left > m_right:
        orientation = "left"
    else:
        orientation = "right"
        
    return orientation

In [21]:
def get_patches(img, mask, img_name, patch_dim=1024, debug=False):
    """
    Compute the patch enclosing the tumor
    Args:
        img(numpy array): the input image of the breast
        mask(numpy array): the mask of the tumor
        img_name(str): image filename
        patch_dim(int): dimension of the output patch, default 1024
        debug(bool): whether to save an image with the patch and the tumor center drawn on the mask
    
    Returns:
        roi_crop(numpy array): cropped input image
        mask_crop(numpy array): cropped input mask
        
    """
    
    # get bbox of the tumor
    x, y, w, h = get_dim_tumor(mask)
    center = get_center_rect(x, y, w, h )
    
    orientation = get_img_orientation(img)
    
    # construct patch around center
    patch_tl_x = int(center[0] - patch_dim * 0.5)
    patch_tl_y = int(center[1] - patch_dim * 0.5)
    patch_br_x = patch_tl_x + patch_dim
    patch_br_y = patch_tl_y + patch_dim

    if patch_tl_x < 0:
        if orientation == 'right':
            # length in pixels of the borders at each side of the image: top, bottom, left, right 
            left_pad = abs(patch_tl_x)
            img = cv2.copyMakeBorder(img, 0, 0, left_pad, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            mask = cv2.copyMakeBorder(mask, 0, 0, left_pad, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        patch_tl_x = 0
            

    if patch_tl_y < 0:
        patch_tl_y = 0

    if patch_br_x > mask.shape[1]:
        if orientation == 'left':
            right_pad = patch_br_x - mask.shape[1]
            img = cv2.copyMakeBorder(img, 0, 0, 0, right_pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            mask = cv2.copyMakeBorder(mask, 0, 0, 0, right_pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        patch_br_x = mask.shape[1]

    if patch_br_y > mask.shape[0]:
        patch_br_y = mask.shape[0]

#     print("Crop rect coord", patch_tl_x, patch_tl_y, patch_br_x, patch_br_y)
    if debug == True:
        mask2 = mask.copy()
        img2 = img.copy()
        preview_folder = pt.Path.cwd()/"preview"
        if not preview_folder.exists():
            os.mkdir(preview_folder)
            
        
        cv2.circle(mask2, (int(center[0]), int(center[1])), 10, (0, 0, 255), -1)
        cv2.rectangle(mask2, (patch_tl_x, patch_tl_y), (patch_br_x, patch_br_y), (0, 0, 255), 3)
        mask_path = preview_folder/"mask_{}.jpg".format(img_name)
        cv2.imwrite(str(mask_path), mask2)
        
        cv2.circle(img2, (int(center[0]), int(center[1])), 10, (0, 0, 255), -1)
        cv2.rectangle(img2, (patch_tl_x, patch_tl_y), (patch_br_x, patch_br_y), (0, 0, 255), 3)
        image_path = preview_folder/"img_{}".format(img_name)
        cv2.imwrite(str(image_path), img2)
        print("[INFO] {}: ".format(img_name))
        print("         bbox:", x, y, w, h)
        print("         patch:", patch_tl_x, patch_tl_y, patch_br_x, patch_br_y)
        
    roi_crop = img[patch_tl_y:patch_br_y, patch_tl_x:patch_br_x]
    mask_crop = mask[patch_tl_y:patch_br_y, patch_tl_x:patch_br_x]
    
    return roi_crop, mask_crop

#### Input paths to the data and the patch dimension

In [13]:
DATA_PATH   = pt.Path(r"D:\work\BreastTumorSegmentation\dataset\original_size")
OUTPUT_PATH = pt.Path(r"D:\work\BreastTumorSegmentation\dataset\cropped_size")
IMGS_DIR    = "images"
MASKS_DIR   = "masks"
IMGS_CSV    = DATA_PATH/"images_original.csv"
MASKS_CSV   = DATA_PATH/"masks_original.csv"
PATCH_DIM   = 1024

In [7]:
imgs_df = pd.read_csv(IMGS_CSV)
masks_df = pd.read_csv(MASKS_CSV)

In [22]:
assert len(imgs_df) == len(masks_df), "The number of images should be equal with the number of masks"

nr_removed = 0

with tqdm(desc="Cropping", unit="img", total=len(imgs_df), leave=True) as pbar:
    for img_row, mask_row in zip(imgs_df.iterrows(), masks_df.iterrows()):
        img_row = img_row[1]
        mask_row = mask_row[1]

        img_filename  = pt.Path(img_row['file_path']).name
        mask_filename = pt.Path(mask_row['file_path']).name
        
        assert img_filename == mask_filename
        
        _img = cv2.imread(img_row['file_path'])
        _mask = cv2.imread(mask_row['file_path'])  
        if abs(_img.shape[0] - _mask.shape[0]) > 1 or abs(_img.shape[1] - _mask.shape[1]) > 1:
            # ignore images and masks that do not have the same size
#             print(mask_filename)
            nr_removed += 1
            continue

        roi_crop, mask_crop = get_patches(_img, _mask, img_filename, patch_dim=PATCH_DIM, debug=False)

        roi_path = OUTPUT_PATH/IMGS_DIR/img_filename
        mask_path = OUTPUT_PATH/MASKS_DIR/mask_filename        

        cv2.imwrite(str(roi_path), roi_crop)
        cv2.imwrite(str(mask_path), mask_crop)
        
        pbar.update()

Cropping:   9%|█████▊                                                              | 305/3568 [01:43<16:29,  3.30img/s]

CALC_P_00353_LEFT_CC_M_2.jpg


Cropping:   9%|█████▊                                                              | 306/3568 [01:44<19:51,  2.74img/s]

CALC_P_00353_LEFT_MLO_M_2.jpg


Cropping:  54%|████████████████████████████████████                               | 1921/3568 [10:45<09:12,  2.98img/s]

...

MASS_P_01908_LEFT_CC_M_1.jpg


Cropping:  98%|█████████████████████████████████████████████████████████████████▍ | 3488/3568 [19:57<00:25,  3.16img/s]

MASS_P_01946_RIGHT_MLO_M_1.jpg
MASS_P_01981_RIGHT_CC_M_1.jpg
MASS_P_01981_RIGHT_MLO_M_1.jpg
MASS_P_01983_LEFT_MLO_M_1.jpg
MASS_P_02033_RIGHT_CC_M_1.jpg
MASS_P_02033_RIGHT_MLO_M_1.jpg
MASS_P_02079_RIGHT_CC_M_1.jpg
MASS_P_02079_RIGHT_MLO_M_1.jpg
MASS_P_02092_LEFT_CC_M_1.jpg


Cropping:  98%|█████████████████████████████████████████████████████████████████▍ | 3488/3568 [20:00<00:27,  2.91img/s]

MASS_P_02092_LEFT_MLO_M_1.jpg
